In [47]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



file = "PSG1_0001.edf"
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()


events, events_id = mne.events_from_annotations(data)

Extracting EDF parameters from C:\Users\parod\OneDrive\Escritorio\Máster IA UV\TFM\PSG1_0001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-47-22744f084981>:9: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(file)
<ipython-input-47-22744f084981>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(file)


Used Annotations descriptions: ['EEG arousal', 'Hypopnea', 'Lights Off', 'Limb Movement', 'Montage:PR, Ref', 'Obstructive Apnea', 'Oximeter Event', 'Oxygen Desaturation', 'Pulse Rate Event', 'Recording Analyzer - Auto-Staging', 'Recording Analyzer - Data Trends', 'Recording Analyzer - ECG', 'Recording Analyzer - Sleep Events', 'Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3', 'Sleep stage R', 'Sleep stage W', 'Snoring', 'Start Recording', 'Started Analyzer - Sleep Events', 'Video Recording ON']


In [48]:
events.shape, events_id

((3014, 3),
 {'EEG arousal': 1,
  'Hypopnea': 2,
  'Lights Off': 3,
  'Limb Movement': 4,
  'Montage:PR, Ref': 5,
  'Obstructive Apnea': 6,
  'Oximeter Event': 7,
  'Oxygen Desaturation': 8,
  'Pulse Rate Event': 9,
  'Recording Analyzer - Auto-Staging': 10,
  'Recording Analyzer - Data Trends': 11,
  'Recording Analyzer - ECG': 12,
  'Recording Analyzer - Sleep Events': 13,
  'Sleep stage N1': 14,
  'Sleep stage N2': 15,
  'Sleep stage N3': 16,
  'Sleep stage R': 17,
  'Sleep stage W': 18,
  'Snoring': 19,
  'Start Recording': 20,
  'Started Analyzer - Sleep Events': 21,
  'Video Recording ON': 22})

In [49]:
tmax = 30. - 1. / data.info['sfreq']  # tmax in included

epochs_train = mne.Epochs(raw=data, events=events,
                          event_id=events_id, tmin=0., tmax=tmax, baseline=None, event_repeated='merge')

print(epochs_train)

Not setting metadata
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
3007 matching events found
No baseline correction applied
0 projection items activated
<Epochs |  3007 events (good & bad), 0 - 29.998 sec, baseline off, ~51 kB, data not loaded,
 'EEG arousal': 19
 'Hypopnea': 23
 'Lights Off': 1
 'Limb Movement': 100
 'Limb Movement/Snoring': 1
 'Montage:PR, Ref/Start Recording': 1
 'Obstructive Apnea': 79
 'Oximeter Event': 1
 'Oximeter Event/Pulse Rate Event': 1
 'Oxygen Desaturation': 47
 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events': 1
 'Sleep stage N1': 25
 'Sleep stage N2': 317
 'Sleep stage N3': 193
 'Sleep stage R': 101
 'Sleep stage W': 226
 'Snoring': 1869
 'Started Analyzer - Sleep Events': 1
 'Video Recording ON': 1>


In [50]:
patatuela = epochs_train.load_data()

Loading data for 3007 events and 15360 original time points ...
1 bad epochs dropped


In [51]:
patatuela.events

array([[      19,        0,       23],
       [      67,        0,       22],
       [     101,        0,       24],
       ...,
       [13194240,        0,       18],
       [13209600,        0,       18],
       [13224960,        0,       18]])

In [52]:
patatuela.event_id

{'EEG arousal': 1,
 'Hypopnea': 2,
 'Lights Off': 3,
 'Limb Movement': 4,
 'Obstructive Apnea': 6,
 'Oximeter Event': 7,
 'Oxygen Desaturation': 8,
 'Sleep stage N1': 14,
 'Sleep stage N2': 15,
 'Sleep stage N3': 16,
 'Sleep stage R': 17,
 'Sleep stage W': 18,
 'Snoring': 19,
 'Started Analyzer - Sleep Events': 21,
 'Video Recording ON': 22,
 'Montage:PR, Ref/Start Recording': 23,
 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events': 24,
 'Limb Movement/Snoring': 25,
 'Oximeter Event/Pulse Rate Event': 26}

In [53]:
id_to_class_dict = {value:key for key, value in patatuela.event_id.items()}
id_to_class_dict

{1: 'EEG arousal',
 2: 'Hypopnea',
 3: 'Lights Off',
 4: 'Limb Movement',
 6: 'Obstructive Apnea',
 7: 'Oximeter Event',
 8: 'Oxygen Desaturation',
 14: 'Sleep stage N1',
 15: 'Sleep stage N2',
 16: 'Sleep stage N3',
 17: 'Sleep stage R',
 18: 'Sleep stage W',
 19: 'Snoring',
 21: 'Started Analyzer - Sleep Events',
 22: 'Video Recording ON',
 23: 'Montage:PR, Ref/Start Recording',
 24: 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events',
 25: 'Limb Movement/Snoring',
 26: 'Oximeter Event/Pulse Rate Event'}

In [54]:
patatuela._data.shape, patatuela.events

((3006, 50, 15360),
 array([[      19,        0,       23],
        [      67,        0,       22],
        [     101,        0,       24],
        ...,
        [13194240,        0,       18],
        [13209600,        0,       18],
        [13224960,        0,       18]]))

In [55]:
objective_stages = ['Sleep stage N1',
'Sleep stage N2',
'Sleep stage N3',
'Sleep stage R',
'Sleep stage W']

In [56]:
epochs_to_df = [a for a in patatuela]
label_to_df = [id_to_class_dict[i] for i in patatuela.events[:,-1]]
dict_to_df = {"Epoch":epochs_to_df,
              "Label":label_to_df}

In [57]:
prueba = {}

epochs_to_df = []
label_to_df = []
for epoch, label in zip(patatuela, patatuela.events[:,-1]):
    # print(label)
    label = id_to_class_dict[label]
    if label in objective_stages:
        # print("True")
        epochs_to_df.append(epoch)
        label_to_df.append(label)
    # else:
        # print("False")

In [58]:
dict_to_df = {"Epoch":epochs_to_df,
              "Label":label_to_df}

In [72]:
df1=pd.DataFrame(dict_to_df)

In [76]:
df1

,Epoch,Label
0,"[[-0.0004177615014877544, -0.00041776150148775...",Sleep stage W
1,"[[-9.278095674067275e-05, -8.421149004348803e-...",Sleep stage W
2,"[[-0.00019297779812314013, -0.0001942961776150...",Sleep stage W
3,"[[-2.356603341725778e-05, -2.191805905241457e-...",Sleep stage W
4,"[[-4.2352941176470415e-05, -4.2352941176470415...",Sleep stage W
...,...,...
856,"[[2.3895628290226768e-05, 2.785076676585048e-0...",Sleep stage W
857,"[[2.191805905241491e-05, 2.5214007782101335e-0...",Sleep stage W
858,"[[1.0382238498512416e-05, 1.33485923552302e-05...",Sleep stage W
859,"[[4.7956054016937685e-05, 4.762645914396904e-0...",Sleep stage W


In [60]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



file2 = "PSG1_0002.edf"
data2 = mne.io.read_raw_edf(file2)
raw_data2 = data2.get_data()


events2, events_id2 = mne.events_from_annotations(data2)

Extracting EDF parameters from C:\Users\parod\OneDrive\Escritorio\Máster IA UV\TFM\PSG1_0002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-60-2f5cf4a5629b>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data2 = mne.io.read_raw_edf(file2)


Used Annotations descriptions: ['Impedance at 10 kOhm', 'Lights On', 'Limb Movement', 'Sleep stage W', 'Stopped Analyzer - Sleep Events']


In [61]:
events2.shape, events_id2

((26, 3),
 {'Impedance at 10 kOhm': 1,
  'Lights On': 2,
  'Limb Movement': 3,
  'Sleep stage W': 4,
  'Stopped Analyzer - Sleep Events': 5})

In [62]:
tmax = 30. - 1. / data.info['sfreq']  # tmax in included

epochs_train2 = mne.Epochs(raw=data2, events=events2,
                          event_id=events_id2, tmin=0., tmax=tmax, baseline=None, event_repeated='merge')

print(epochs_train2)

Not setting metadata
Not setting metadata
26 matching events found
No baseline correction applied
0 projection items activated
<Epochs |  26 events (good & bad), 0 - 29.998 sec, baseline off, ~51 kB, data not loaded,
 'Impedance at 10 kOhm': 1
 'Lights On': 1
 'Limb Movement': 2
 'Sleep stage W': 21
 'Stopped Analyzer - Sleep Events': 1>


In [63]:

patatuela2 = epochs_train2.load_data()

Loading data for 26 events and 15360 original time points ...
3 bad epochs dropped


In [64]:
patatuela2.events

array([[   176,      0,      1],
       [  9042,      0,      4],
       [ 24402,      0,      4],
       [ 39762,      0,      4],
       [ 48154,      0,      3],
       [ 55122,      0,      4],
       [ 70482,      0,      4],
       [ 82584,      0,      3],
       [ 85842,      0,      4],
       [101202,      0,      4],
       [116562,      0,      4],
       [131922,      0,      4],
       [147282,      0,      4],
       [162642,      0,      4],
       [178002,      0,      4],
       [193362,      0,      4],
       [208722,      0,      4],
       [224082,      0,      4],
       [239442,      0,      4],
       [254802,      0,      4],
       [270162,      0,      4],
       [285522,      0,      4],
       [300882,      0,      4]])

In [65]:
patatuela2.event_id

{'Impedance at 10 kOhm': 1,
 'Lights On': 2,
 'Limb Movement': 3,
 'Sleep stage W': 4,
 'Stopped Analyzer - Sleep Events': 5}

In [66]:
id_to_class_dict2 = {value:key for key, value in patatuela2.event_id.items()}
id_to_class_dict2

{1: 'Impedance at 10 kOhm',
 2: 'Lights On',
 3: 'Limb Movement',
 4: 'Sleep stage W',
 5: 'Stopped Analyzer - Sleep Events'}

In [67]:
patatuela2._data.shape, patatuela2.events

((23, 50, 15360),
 array([[   176,      0,      1],
        [  9042,      0,      4],
        [ 24402,      0,      4],
        [ 39762,      0,      4],
        [ 48154,      0,      3],
        [ 55122,      0,      4],
        [ 70482,      0,      4],
        [ 82584,      0,      3],
        [ 85842,      0,      4],
        [101202,      0,      4],
        [116562,      0,      4],
        [131922,      0,      4],
        [147282,      0,      4],
        [162642,      0,      4],
        [178002,      0,      4],
        [193362,      0,      4],
        [208722,      0,      4],
        [224082,      0,      4],
        [239442,      0,      4],
        [254802,      0,      4],
        [270162,      0,      4],
        [285522,      0,      4],
        [300882,      0,      4]]))

In [68]:
objective_stages2 = ['Sleep stage W']

In [69]:
epochs_to_df2 = [a for a in patatuela2]
label_to_df2 = [id_to_class_dict2[i] for i in patatuela2.events[:,-1]]
dict_to_df2 = {"Epoch":epochs_to_df2,
              "Label":label_to_df2}

In [70]:
prueba = {}

epochs_to_df = []
label_to_df = []
for epoch, label in zip(patatuela2, patatuela2.events[:,-1]):
    # print(label)
    label = id_to_class_dict2[label]
    if label in objective_stages2:
        # print("True")
        epochs_to_df2.append(epoch)
        label_to_df2.append(label)
    # else:
        # print("False")

In [71]:
dict_to_df2 = {"Epoch":epochs_to_df2,
              "Label":label_to_df2}

In [78]:
df2=pd.DataFrame(dict_to_df2)
df2=df2[df2.Label=='Sleep stage W']

In [81]:
df1=df1.sort_values(by='Label',ascending=False)

In [92]:
resto=df1[df1.Label!='Sleep stage W'].reset_index(drop=True)

In [85]:
w=df1[df1.Label=='Sleep stage W']

In [88]:
df2.shape

(40, 2)

In [107]:
df1.Epoch.values[0]

50

In [87]:
w

,Epoch,Label
0,"[[-0.0004177615014877544, -0.00041776150148775...",Sleep stage W
288,"[[6.542458228427574e-05, 6.377660791943253e-05...",Sleep stage W
227,"[[4.5319295033188544e-05, 4.894483863584361e-0...",Sleep stage W
228,"[[1.2359807736324273e-05, 8.734264133669202e-0...",Sleep stage W
229,"[[3.3124284733348766e-05, 1.7962920576791197e-...",Sleep stage W
...,...,...
22,"[[1.33485923552302e-05, 7.086289768825989e-06,...",Sleep stage W
18,"[[2.1588464179446268e-05, 2.916914625772505e-0...",Sleep stage W
19,"[[1.598535133897934e-05, 8.075074387731916e-06...",Sleep stage W
20,"[[0.00027076218814374014, 0.000238132295719844...",Sleep stage W


In [91]:
w=w.append(df2).reset_index(drop=True)

In [146]:
defi1=w.append(resto).reset_index(drop=True)
defi1

,Epoch,Label
0,"[[-0.0004177615014877544, -0.00041776150148775...",Sleep stage W
1,"[[6.542458228427574e-05, 6.377660791943253e-05...",Sleep stage W
2,"[[4.5319295033188544e-05, 4.894483863584361e-0...",Sleep stage W
3,"[[1.2359807736324273e-05, 8.734264133669202e-0...",Sleep stage W
4,"[[3.3124284733348766e-05, 1.7962920576791197e-...",Sleep stage W
...,...,...
896,"[[3.180590524147419e-05, 3.3783474479286046e-0...",Sleep stage N1
897,"[[4.7956054016937685e-05, 5.2240787365530035e-...",Sleep stage N1
898,"[[2.2906843671320838e-05, 2.8180361638819122e-...",Sleep stage N1
899,"[[1.0052643625543773e-05, 1.730373083085391e-0...",Sleep stage N1


In [147]:
valores=defi1.Epoch.values

In [187]:
from collections import OrderedDict

# class DefaultListOrderedDict(OrderedDict):
#     def __missing__(self,k):
#         self[k] = []
#         return self[k]

# # keys=['blk','pri','ani']
# # vals1=['blocking','primary','anim']
# # vals2=['S1','S2','S3']
# dic = DefaultListOrderedDict()
# for i,key in enumerate(channels):
#     dic[key].append(valores[i])
# #     dic[key].append(vals2[i])

# print (dic)

In [138]:
channels = data.ch_names

In [188]:
diccionarios=[dict(zip(channels, sublist)) for sublist in valores]

In [205]:
c3=[d['C3'] for d in diccionarios]
c4=[d['C4'] for d in diccionarios]
o1=[d['O1'] for d in diccionarios]
o2=[d['O2'] for d in diccionarios]
loc=[d['LOC'] for d in diccionarios]
roc=[d['ROC'] for d in diccionarios]
chin=[d['CHIN1'] for d in diccionarios]

In [206]:
c3

[array([-4.17761501e-04, -4.17761501e-04, -4.14795148e-04, ...,
        -9.47585260e-05, -1.02668803e-04, -9.67360952e-05]),
 array([6.54245823e-05, 6.37766079e-05, 6.60837720e-05, ...,
        3.64202335e-05, 3.37834745e-05, 3.08171206e-05]),
 array([4.53192950e-05, 4.89448386e-05, 5.02632181e-05, ...,
        8.73426413e-06, 1.07118334e-05, 1.23598077e-05]),
 array([1.23598077e-05, 8.73426413e-06, 8.73426413e-06, ...,
        5.52071412e-05, 2.35660334e-05, 2.58731975e-05]),
 array([3.31242847e-05, 1.79629206e-05, 1.20302129e-05, ...,
        2.19180591e-05, 2.48844129e-05, 2.81803616e-05]),
 array([3.01579309e-05, 3.34538796e-05, 3.21355001e-05, ...,
        1.33485924e-05, 1.92813001e-05, 5.10872053e-06]),
 array([-2.19180591e-05, -1.53261616e-05, -1.40077821e-05, ...,
         3.11467155e-05,  3.08171206e-05,  2.94987411e-05]),
 array([2.52140078e-05, 2.09292744e-05, 2.29068437e-05, ...,
        3.18059052e-05, 2.94987411e-05, 2.68619821e-05]),
 array([2.58731975e-05, 2.81803616e-

In [207]:
c4

[array([-0.00035942, -0.00035217, -0.00035613, ..., -0.00017155,
        -0.00018012, -0.00017551]),
 array([5.52071412e-05, 5.35591669e-05, 5.25703822e-05, ...,
        2.15884642e-05, 1.79629206e-05, 1.46669718e-05]),
 array([-9.72304875e-06, -9.06385901e-06, -6.42710002e-06, ...,
        -1.41560998e-04, -1.41890593e-04, -1.43208972e-04]),
 array([-1.45516136e-04, -1.52767224e-04, -1.57051957e-04, ...,
        -6.09750515e-06,  1.64797436e-07, -6.75669490e-06]),
 array([-1.66445411e-05, -2.65323873e-05, -2.65323873e-05, ...,
        -1.17006180e-05, -7.74547951e-06, -3.13115129e-06]),
 array([-8.23987182e-07,  3.46074617e-06,  3.46074617e-06, ...,
        -5.92446784e-04, -5.89150835e-04, -5.96072328e-04]),
 array([-6.28372625e-04, -6.29361410e-04, -6.24087892e-04, ...,
         2.14236667e-06,  1.64797436e-07, -8.23987182e-07]),
 array([-6.75669490e-06, -1.07118334e-05, -1.07118334e-05, ...,
        -6.42710002e-06, -9.39345388e-06, -1.40077821e-05]),
 array([-1.36781872e-05, -1.26

In [220]:
defi2=pd.DataFrame(np.hstack((c3,c4,o1,o2,loc,roc,chin)),columns=[f'col{i}' for i in range(107520)])
defi2['Target']=defi1.Label

In [221]:
defi2

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col107511,col107512,col107513,col107514,col107515,col107516,col107517,col107518,col107519,Target
0,-0.000418,-0.000418,-0.000415,-0.000419,-0.000415,-0.000402,-3.923827e-04,-0.000417,-0.000438,-0.000441,...,-4.136416e-05,-0.000029,-0.000015,-1.730373e-05,-0.000012,-0.000017,-0.000048,-0.000056,-0.000055,Sleep stage W
1,0.000065,0.000064,0.000066,0.000062,0.000059,0.000053,4.927443e-05,0.000051,0.000052,0.000054,...,3.147631e-05,0.000027,0.000028,2.982834e-05,0.000029,0.000032,0.000035,0.000034,0.000034,Sleep stage W
2,0.000045,0.000049,0.000050,0.000049,0.000046,0.000048,4.894484e-05,0.000047,0.000041,0.000044,...,-5.108721e-06,-0.000004,-0.000017,-2.554360e-05,-0.000020,-0.000012,-0.000017,-0.000016,-0.000010,Sleep stage W
3,0.000012,0.000009,0.000009,0.000010,0.000012,0.000012,1.104143e-05,0.000008,0.000007,0.000004,...,2.125887e-05,0.000040,0.000056,5.817350e-05,0.000033,0.000028,0.000003,-0.000034,-0.000023,Sleep stage W
4,0.000033,0.000018,0.000012,0.000019,0.000027,0.000030,3.477226e-05,0.000036,0.000034,0.000032,...,8.075074e-06,0.000006,0.000003,9.063859e-06,0.000015,0.000017,0.000015,0.000015,0.000010,Sleep stage W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,0.000032,0.000034,0.000037,0.000042,0.000042,0.000039,3.674983e-05,0.000034,0.000029,0.000028,...,-8.239872e-07,0.000003,0.000001,8.239872e-07,0.000004,0.000005,-0.000002,-0.000002,-0.000008,Sleep stage N1
897,0.000048,0.000052,0.000050,0.000047,0.000043,0.000040,3.378347e-05,0.000034,0.000037,0.000037,...,8.734264e-06,0.000018,0.000022,2.488441e-05,0.000025,0.000029,0.000023,0.000020,0.000012,Sleep stage N1
898,0.000023,0.000028,0.000031,0.000030,0.000029,0.000028,2.455482e-05,0.000024,0.000027,0.000028,...,8.025635e-05,0.000083,0.000080,7.926757e-05,0.000085,0.000085,0.000084,0.000080,0.000077,Sleep stage N1
899,0.000010,0.000017,0.000019,0.000015,0.000011,0.000004,8.239872e-07,0.000003,0.000008,0.000012,...,8.075074e-06,-0.000002,-0.000017,-7.745480e-06,-0.000008,-0.000006,-0.000007,-0.000005,0.000008,Sleep stage N1
